In [1]:
# 导包和拿数据集
import numpy as np
from math import sqrt
from collections import Counter
from sklearn.datasets import load_iris
iris = load_iris()
logits = iris.data
labels = iris.target
print(logits[:2])
print(labels[:2])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]]
[0 0]


In [2]:
# 常规操作，划分数据集
def train_test_split(X , y , test_ratio = 0.2 , seed = None):
    assert X.shape[0] == y.shape[0] , '特征和标签的长度要相同'
    assert 0 <= test_ratio <= 1 , 'test_ratio参数要符合规范'
    
    # 随机种子
    if seed:
        np.random.seed(seed)
        
    shuffled_indexes = np.random.permutation(len(X))
    test_size = int(len(y) * test_ratio)
    test_indexes = shuffled_indexes[:test_size]
    train_indexes = shuffled_indexes[test_size:]
    
    X_train = X[train_indexes]
    y_train = y[train_indexes]
    X_test = X[test_indexes]
    y_test = y[test_indexes]
    
    return X_train , X_test , y_train , y_test

X_train , X_test , y_train , y_test = train_test_split(logits , labels , test_ratio = 0.2)

In [3]:
# 封装KNN
class KNNClassifier:

    def __init__(self, k):
        """初始化kNN分类器"""
        assert k >= 1, "k must be valid"
        self.k = k
        self._X_train = None
        self._y_train = None

    def fit(self, X_train, y_train):
        """根据训练数据集X_train和y_train训练kNN分类器"""
        assert X_train.shape[0] == y_train.shape[0], \
            "the size of X_train must be equal to the size of y_train"
        assert self.k <= X_train.shape[0], \
            "the size of X_train must be at least k."

        self._X_train = X_train
        self._y_train = y_train
        return self

    def predict(self, X_predict):
        """给定待预测数据集X_predict，返回表示X_predict的结果向量"""
        assert self._X_train is not None and self._y_train is not None, \
                "must fit before predict!"
        assert X_predict.shape[1] == self._X_train.shape[1], \
                "the feature number of X_predict must be equal to X_train"

        y_predict = [self._predict(x) for x in X_predict]
        return np.array(y_predict)

    def _predict(self, x):
        """给定单个待预测数据x，返回x的预测结果值"""
        assert x.shape[0] == self._X_train.shape[1], \
            "the feature number of x must be equal to X_train"

        distances = [sqrt(np.sum((x_train - x) ** 2))
                     for x_train in self._X_train]
        nearest = np.argsort(distances)

        topK_y = [self._y_train[i] for i in nearest[:self.k]]
        votes = Counter(topK_y)

        return votes.most_common(1)[0][0]

    def score(self, X_test, y_test):
        """根据测试数据集 X_test 和 y_test 确定当前模型的准确度"""

        y_predict = self.predict(X_test)
        return np.sum(y_test == y_predict) / len(y_test)

    def __repr__(self):
        return "KNN(k=%d)" % self.k

In [4]:
# 使用KNN分类模型
KNN = KNNClassifier(k = 3)
KNN.fit(X_train , y_train)
print(KNN.predict(X_test)[:10])
print(KNN.score(X_test , y_test))

[2 1 2 2 0 2 2 2 2 1]
0.9666666666666667
